In [0]:
import keras
from keras.models import Sequential, load_model
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
vgg16_model = VGG16()

In [0]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [0]:
model.summary()

In [0]:
for layer in model.layers:
    layer.trainable = False

In [0]:
model.add(Dense(2, activation='softmax'))

In [0]:
model.summary()

In [0]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical')

model.fit_generator(
        train_set,
        steps_per_epoch=100,
        epochs=100,
        validation_data=test_set,
        validation_steps=10,
        callbacks=[checkpoint,early])

model.save('Vgg16_saved.h5')

In [0]:
y_pred = []
y_crt = []
def test_accuracy(model, test_set):
    num_correct = 0
    num_guesses = 0
    for i in range(len(test_set)):
        a = test_set[i]
        guess = model.predict(a[0])
        correct = a[1]
        if(round(guess[0][0])==1):
          y_pred.append(0)
        else:
          y_pred.append(1)
        if(correct[0][0]==1):
          y_crt.append(0)
        else:
          y_crt.append(1)

test_accuracy(model, test_set)
cnf_hm = sns.heatmap(confusion_matrix(y_crt, y_pred), cmap="YlGnBu", annot=True)
print(cnf_hm)